# Exampville Mode Choice

Discrete choice modeling is at the heart of many transportion planning models.
In this example, we will examine the development of a mode choice model for 
Exampville, an entirely fictional town built for the express purpose of 
demostrating the use of discrete choice modeling tools for transportation 
planning.

In this notebook, we will walk through the creation of a tour mode choice model.
This example will assume the reader is familiar with the mathematical basics of discrete choice
modeling generally, and will focus on the technical aspects of estimating the parameters
of a discrete choice model in Python using [Larch](https://larch.newman.me).

In [1]:
import larch, numpy, pandas, os

To begin, we'll load some raw data. The Exampville data 
contains a set of files similar to what we might find for 
a real travel survey: network skims, and tables of households, 
persons, and tours.

In [2]:
import larch.exampville

The skims data is a file in [openmatrix](https://github.com/osPlanning/omx/wiki) format, which contains a 
series of two-dimensional arrays describing zone-to-zone transportation
level-of-service attributes.  We can see below that this file contains 
data on travel times and costs by auto, transit, biking, and walking, for
travel to and from each of 40 travel analysis zones in Exampville.
Ideally, these skim values represent the "observed" travel times and costs
for trips between each pair of zones, but generally these matrixes are
approximations of these real values generated by a base-case transportation
model.

In [3]:
skims = larch.OMX( larch.exampville.files.skims, mode='r' )
skims

<larch.OMX> ⋯/exampville_skims.omx
 |  shape:(40, 40)
 |  data:
 |    AUTO_COST    (float64)
 |    AUTO_DIST    (float64)
 |    AUTO_TIME    (float64)
 |    BIKE_TIME    (float64)
 |    TRANSIT_FARE (float64)
 |    TRANSIT_IVTT (float64)
 |    TRANSIT_OVTT (float64)
 |    WALK_DIST    (float64)
 |    WALK_TIME    (float64)
 |  lookup:
 |    TAZ_AREA_TYPE (40 |S3)
 |    TAZ_ID        (40 int64)

The other files are simple `csv` text files, containing row-wise 
data about households, persons, and tours, as might be contained in survey 
results from a household travel survey conducted in Exampville.

In [4]:
hh = pandas.read_csv( larch.exampville.files.hh )
pp = pandas.read_csv( larch.exampville.files.person )
tour = pandas.read_csv( larch.exampville.files.tour )

Let's check out what's in each table.

In [5]:
hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   X            5000 non-null   float64
 1   Y            5000 non-null   float64
 2   INCOME       5000 non-null   float64
 3   N_VEHICLES   5000 non-null   int64  
 4   HHSIZE       5000 non-null   int64  
 5   geometry     5000 non-null   object 
 6   HOMETAZ      5000 non-null   int64  
 7   HHID         5000 non-null   int64  
 8   N_TRIPS      5000 non-null   int64  
 9   N_TRIPS_HBW  5000 non-null   int64  
 10  N_TRIPS_HBO  5000 non-null   int64  
 11  N_TRIPS_NHB  5000 non-null   int64  
 12  N_WORKERS    5000 non-null   int64  
dtypes: float64(3), int64(9), object(1)
memory usage: 507.9+ KB


In [6]:
pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12349 entries, 0 to 12348
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   PERSONID       12349 non-null  int64
 1   HHID           12349 non-null  int64
 2   HHIDX          12349 non-null  int64
 3   AGE            12349 non-null  int64
 4   WORKS          12349 non-null  int64
 5   N_WORK_TOURS   12349 non-null  int64
 6   N_OTHER_TOURS  12349 non-null  int64
 7   N_TOURS        12349 non-null  int64
 8   N_TRIPS        12349 non-null  int64
 9   N_TRIPS_HBW    12349 non-null  int64
 10  N_TRIPS_HBO    12349 non-null  int64
 11  N_TRIPS_NHB    12349 non-null  int64
dtypes: int64(12)
memory usage: 1.1 MB


In [7]:
tour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20739 entries, 0 to 20738
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   TOURID       20739 non-null  int64
 1   HHID         20739 non-null  int64
 2   PERSONID     20739 non-null  int64
 3   DTAZ         20739 non-null  int64
 4   TOURMODE     20739 non-null  int64
 5   TOURPURP     20739 non-null  int64
 6   N_STOPS      20739 non-null  int64
 7   N_TRIPS      20739 non-null  int64
 8   N_TRIPS_HBW  20739 non-null  int64
 9   N_TRIPS_HBO  20739 non-null  int64
 10  N_TRIPS_NHB  20739 non-null  int64
dtypes: int64(11)
memory usage: 1.7 MB


## Preprocessing

To use these data tables for mode choice modeling, we'll need to 
filter them so they only include relevant rows, and merge
them into a unified composite dataset.

### Filtering

Mode choice models are often created seperately for each tour purpose. 
We can review the purposes contained in our data by using the `statistics`
method, which Larch adds to pandas.Series objects:

In [8]:
tour.TOURPURP.statistics()

┣               n: 20739
┣         minimum: 1
┣         maximum: 2
┣          median: 2.0
┣       histogram: <xmle.Elem 'div' with 1 children>
┣            mean: 1.6352765321375187
┣           stdev: 0.48135253178190196
┣           zeros: 0
┣       positives: 20739
┣       negatives: 0
┣ nonzero_minimum: 1
┣ nonzero_maximum: 2
┣    nonzero_mean: 1.6352765321375187
┣   nonzero_stdev: 0.48135253178190196

As we can see above, in Exampville, there are only two purposes for tours.  These
purposes are defined as: 

- work (purpose=1) and 
- non-work (purpose=2). 

We want to first estimate a mode choice model for work tours, 
so we’ll begin by creating a working dataframe,
filtering the tours data to exclude non-work tours:

In [9]:
df = tour[tour.TOURPURP == 1]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7564 entries, 0 to 20736
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   TOURID       7564 non-null   int64
 1   HHID         7564 non-null   int64
 2   PERSONID     7564 non-null   int64
 3   DTAZ         7564 non-null   int64
 4   TOURMODE     7564 non-null   int64
 5   TOURPURP     7564 non-null   int64
 6   N_STOPS      7564 non-null   int64
 7   N_TRIPS      7564 non-null   int64
 8   N_TRIPS_HBW  7564 non-null   int64
 9   N_TRIPS_HBO  7564 non-null   int64
 10  N_TRIPS_NHB  7564 non-null   int64
dtypes: int64(11)
memory usage: 709.1 KB


### Merging

We can then merge data from the three survey tables using the usual 
`pandas` syntax for merging.

In [11]:
df = df.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))

Merging the skims data is more complicated, as we want to select not only
the correct row, but also the correct column for each observation.  We could do this
by transforming the skims data in such a way that every origin-destination
pair is on its own row, but for very large zone systems this can be 
inefficient.  Larch provides a more efficient method to directly extract
a DataFrame with the right information, based on the two-dimensional structure
of the skims.

Our zone numbering system starts with zone 1, as is common for many TAZ numbering 
systems seen in practice.  But, for looking up data in the skims matrix using Larch, 
we'll need to use zero-based numbering that is standard in Python.  So we'll create two new 
TAZ-index columns to assist this process.

In [12]:
df["HOMETAZi"] = df["HOMETAZ"] - 1
df["DTAZi"] = df["DTAZ"] - 1

For this tour mode choice model, we can pick values 
out of the skims for the known O-D of the tour, so 
that we have access to the level-of-service data for
each possible mode serving that O-D pair.  We'll 
use the `get_rc_dataframe` method of the `larch.OMX`
object, which lets us give the a list of the index for the 
production and attraction (row and column) value we
want to use.  

In [13]:
los_data = skims.get_rc_dataframe(
    df["HOMETAZi"], df["DTAZi"],
)
los_data

,AUTO_COST,AUTO_DIST,AUTO_TIME,BIKE_TIME,TRANSIT_FARE,TRANSIT_IVTT,TRANSIT_OVTT,WALK_DIST,WALK_TIME
0,0.588461,1.681318,5.043955,8.406591,0.0,0.000000,2.241758,1.681318,33.626365
1,1.925965,5.502758,16.508274,16.843315,2.5,1.535243,47.830632,3.368663,67.373260
2,0.621348,1.775280,5.325840,8.876400,0.0,0.000000,35.505600,1.775280,35.505600
3,0.480560,1.373029,9.119086,6.865143,0.0,0.000000,27.460572,1.373029,27.460572
4,0.480560,1.373029,9.119086,6.865143,0.0,0.000000,27.460572,1.373029,27.460572
...,...,...,...,...,...,...,...,...,...
7559,0.990705,2.830584,4.842947,8.820128,2.5,1.652554,21.496711,1.764026,35.280512
7560,1.066707,3.047736,7.201386,15.238678,2.5,3.746109,11.138607,3.047736,60.954713
7561,1.294075,3.697356,7.290231,18.486780,2.5,10.514265,33.910995,3.697356,73.947121
7562,1.066707,3.047736,7.201386,15.238678,2.5,3.746109,11.138607,3.047736,60.954713


In [14]:
df = df.join(los_data)

We can review the `df` frame to see what variables are now included.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7564 entries, 0 to 7563
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TOURID         7564 non-null   int64  
 1   HHID           7564 non-null   int64  
 2   PERSONID       7564 non-null   int64  
 3   DTAZ           7564 non-null   int64  
 4   TOURMODE       7564 non-null   int64  
 5   TOURPURP       7564 non-null   int64  
 6   N_STOPS        7564 non-null   int64  
 7   N_TRIPS_x      7564 non-null   int64  
 8   N_TRIPS_HBW_x  7564 non-null   int64  
 9   N_TRIPS_HBO_x  7564 non-null   int64  
 10  N_TRIPS_NHB_x  7564 non-null   int64  
 11  X              7564 non-null   float64
 12  Y              7564 non-null   float64
 13  INCOME         7564 non-null   float64
 14  N_VEHICLES     7564 non-null   int64  
 15  HHSIZE         7564 non-null   int64  
 16  geometry       7564 non-null   object 
 17  HOMETAZ        7564 non-null   int64  
 18  N_TRIPS_

In [16]:
# For clarity, we can define numbers as names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

In [17]:
dfs = larch.DataFrames(
    co=df, 
    alt_codes=[DA,SR,Walk,Bike,Transit], 
    alt_names=['DA','SR','Walk','Bike','Transit'],
    ch_name='TOURMODE',
)

## Model Definition

Now we are ready to create our model.  We'll create a `larch.Model` object
to do so, and link it to the data we just created.

In [18]:
m = larch.Model(dataservice=dfs)
m.title = "Exampville Work Tour Mode Choice v1"

We will explicitly define the set of utility functions 
we want to use.  Because the DataFrames we are using to 
serve data to this model contains exclusively `idco` format
data, we'll use the `utility_co` mapping, which allows us
to define a unique utility function for each alternative.

Each utility function must be expressed as a linear-in-parameters
function to combine raw or pre-computed data values with 
named model parameters.  To facilitate writing these functions,
Larch provides two special classes: parameter references (`P`) and
data references (`X`).

In [19]:
from larch import P, X

Parameter and data references can be defined using either a function-like notation,
or a attribute-like notation.

In [20]:
P('NamedParameter')

P.NamedParameter

In [21]:
X.NamedDataValue

X.NamedDataValue

In either case, if the named value contains any spaces or non-alphanumeric characters,
it must be given in function-like notation only, as Python will not accept
those characters in the attribute-like form.

In [22]:
P('Named Parameter')

P('Named Parameter')

Data references can name an exact column that appears in the `DataFrames` as
defined above, or can include simple transformations of that data, so long
as these transformations can be done without regard to any estimated parameter.
For example, we can use the log of income:

In [23]:
X("log(INCOME)")

X('log(INCOME)')

To write a linear-in-parameters utility function, we simply multiply together
a parameter reference and a data reference, and then optionally add that
to one or more similar terms.

In [24]:
P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

It is permissible to omit the data reference on a term 
(in which case it is implicitly set to 1.0).

In [25]:
P.ASC + P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

P.ASC + P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST

We can then combine these to write utility functions for each
alternative in the Exampville data:

In [26]:
m.utility_co[DA] = (
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * X.AUTO_COST # dollars per mile
)

m.utility_co[SR] = (
        + P.ASC_SR
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * (X.AUTO_COST * 0.5) # dollars per mile, half share
        + P("LogIncome:SR") * X("log(INCOME)")
)

m.utility_co[Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

To write a nested logit model, we'll attach some nesting nodes to the 
model's `graph`.  Each `new_node` allows us to define the set of 
codes for the child nodes (elemental alternatives, or lower level nests)
as well as giving the new nest a name and assigning a logsum parameter.
The return value of this method is the node code for the newly created 
nest, which then can potenially be used as a child code when creating
a higher level nest.  We do this below, adding the 'Car' nest into the 
'Motor' nest.

In [27]:
Car = m.graph.new_node(parameter='Mu:Car', children=[DA,SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Walk,Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car,Transit], name='Motor')

Let's visually check on the nesting structure.

In [28]:
m.graph

The tour mode choice model's choice variable is indicated by 
the code value in 'TOURMODE', and this can be 
defined for the model using `choice_co_code`.

In [29]:
m.choice_co_code = 'TOURMODE'

We can also give a dictionary of availability conditions based 
on values in the `idco` data, using the `availability_co_vars`
attribute.  Alternatives that are always available can be indicated
by setting the criterion to 1.  For alternative that are only sometimes
available, we can give an availability criteria in the same manner as
for a data reference described above: either by giving the name of 
a variable in the data, or an expression that can be evaluated using
the data alone.  In the case of availability criteria, these will be
tranformed to boolean (true/false) values, so data that evaluates as
0 will be unavailable, and data that evaluates as non-zero will be 
available (including, perhaps counterintuitively, negative numbers).

In [30]:
m.availability_co_vars = {
    DA: 'AGE >= 16',
    SR: 1,
    Walk: 'WALK_TIME < 60',
    Bike: 'BIKE_TIME < 60',
    Transit: 'TRANSIT_FARE>0',
}

Then let's prepare this data for estimation.  Even though the
data is already in memory, the `load_data` method is used to 
pre-process the data, extracting the required values, pre-computing 
the values of fixed expressions, and assembling the results into
contiguous arrays suitable for computing the log likelihood values
efficiently.

## Model Estimation

In [31]:
m.load_data()

We can check on some important statistics of this loaded and preprocessed data even
before we estimate the model.

In [32]:
m.dataframes.choice_avail_summary()

,name,chosen,available
1,DA,6052.0,7564
2,SR,810.0,7564
3,Walk,196.0,4179
4,Bike,72.0,7564
5,Transit,434.0,4199
< Total All Alternatives >,,7564.0,


In [33]:
m.dataframes.data_co.statistics()

AUTO_COST,7564,0.194926,4.30796,1.00945,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gMBgAAAAAAAAYFAAAAAAAArwUAAAAAAABzAwAAAAAAANwCAAAAAAAATgIAAAAAAADJAQAAAAAAALcAAAAAAAAAFQEAAAAAAABhAAAAAAAAACAAAAAAAAAAFQAAAAAAAAADAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcCfSia1S88g/iwubY61c4D+MotMbhnzqP8YcBmovTvI/R2gixhte9z/Hsz4iCG78P6R/LT/6vgBAZKU7bfBGA0Aky0mb5s4FQOTwV8ncVghApBZm99LeCkBlPHQlyWYNQCViglO/7g9A80PIwFo7EUCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",1.20601,0.754844,0,7564,0,0.194926,4.30796,1.20601,0.754844
AUTO_COST*(0.5),7564,0.0974628,2.15398,0.504724,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gMBgAAAAAAAAYFAAAAAAAArwUAAAAAAABzAwAAAAAAANwCAAAAAAAATgIAAAAAAADJAQAAAAAAALcAAAAAAAAAFQEAAAAAAABhAAAAAAAAACAAAAAAAAAAFQAAAAAAAAADAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcCfSia1S87g/iwubY61c0D+MotMbhnzaP8YcBmovTuI/R2gixhte5z/Hsz4iCG7sP6R/LT/6vvA/ZKU7bfBG8z8ky0mb5s71P+TwV8ncVvg/pBZm99Le+j9lPHQlyWb9PyViglO/7v8/80PIwFo7AUCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",0.603005,0.377422,0,7564,0,0.0974628,2.15398,0.603005,0.377422
AUTO_TIME,7564,0.930008,29.4415,7.61571,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gxAwAAAAAAAKoFAAAAAAAAgwUAAAAAAAD3BQAAAAAAAGADAAAAAAAAagIAAAAAAABfAQAAAAAAACMBAAAAAAAAmwAAAAAAAAAyAAAAAAAAABYAAAAAAAAABgAAAAAAAAACAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcAjW8M2gwu0/O5faV1H8CEB6fBw+/UMVQFetS9DRCR5AGm89MdNnI0CIB1V6vconQPafbMOnLSxAMhxCBklIMEBp6M0qvnkyQKC0WU8zqzRA14Dlc6jcNkAOTXGYHQ45QEYZ/bySPztAfOWI4QdxPUCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",8.22287,4.58134,0,7564,0,0.930008,29.4415,8.22287,4.58134
BIKE_TIME,7564,2.78465,52.2321,13.5864,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gbBAAAAAAAAJwFAAAAAAAAxgUAAAAAAABsAwAAAAAAAGoDAAAAAAAARwIAAAAAAAAQAgAAAAAAAEIBAAAAAAAAwwAAAAAAAAB/AAAAAAAAAD0AAAAAAAAAIAAAAAAAAAABAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcMjEqIj3RgZAPICxxGpaGkAKT4firMgkQPXdtWIkZCxAcDZy8c3/MUDmfYmxic01QFzFoHFFmzlA0gy4MQFpPUAkqud4XptAQN5N81g8gkJAmvH+OBppREBUlQoZ+E9GQBA5FvnVNkhAytwh2bMdSkCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",15.9827,9.29732,0,7564,0,2.78465,52.2321,15.9827,9.29732
TRANSIT_FARE,7564,0,2.5,2.5,Histograms are purple if the data is represented as discrete values.,1.38782,1.24238,3365,4199,0,2.5,2.5,2.5,0
TRANSIT_IVTT,7564,0,12.1668,1.44769,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////Sw

If we are satisfied with the statistics we see above, we
can go ahead and estimate the model.  Estimation is done
using maximium likelihood techniques, relying on the `scipy.optimize`
library for providing a variety of algorithms for solving 
this non-linear optimization problem.
For nested logit models, the 'SLSQP' method often works well.

In [34]:
result = m.maximize_loglike(method='slsqp')

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_Bike,-0.257901,0.0,0.0,-inf,inf,0,,-0.257901
ASC_SR,1.423159,0.0,0.0,-inf,inf,0,,1.423159
ASC_Transit,6.754794,0.0,0.0,-inf,inf,0,,6.754794
ASC_Walk,8.621893,0.0,0.0,-inf,inf,0,,8.621893
Cost,-0.175720,0.0,0.0,-inf,inf,0,,-0.175720
InVehTime,-0.123723,0.0,0.0,-inf,inf,0,,-0.123723
LogIncome:Bike,-0.196995,0.0,0.0,-inf,inf,0,,-0.196995
LogIncome:SR,-0.193845,0.0,0.0,-inf,inf,0,,-0.193845
LogIncome:Transit,-0.557177,0.0,0.0,-inf,inf,0,,-0.557177
LogIncome:Walk,-0.522822,0.0,0.0,-inf,inf,0,,-0.522822


After we find the best fitting parameters, we can compute
some variance-covariance statistics, incuding standard errors of
the estimates and t statistics, using `calculate_parameter_covariance`.

In [35]:
m.calculate_parameter_covariance()

Then we can review the results in a variety of report tables.

In [36]:
m.parameter_summary()

Parameter              Value  Std Err  t Stat  Null Value
     ASC_Bike            -0.2579     1.34   -0.19         0.0
       ASC_SR              1.423     1.00    1.42         0.0
  ASC_Transit              6.755     2.06    3.27         0.0
     ASC_Walk              8.622     1.14    7.57         0.0
         Cost            -0.1757    0.120   -1.47         0.0
    InVehTime            -0.1237   0.0292   -4.24         0.0
    LogIncome      Bike   -0.197    0.124   -1.59         0.0
                     SR  -0.1938    0.135   -1.43         0.0
                Transit  -0.5572    0.169   -3.29         0.0
                   Walk  -0.5228    0.100   -5.21         0.0
           Mu       Car   0.2593    0.181   -4.10         1.0
                  Motor   0.8017    0.201   -0.99         1.0
               NonMotor   0.8538    0.112   -1.30         1.0
 NonMotorTime            -0.2656   0.0163  -16.29         0.0
   OutVehTime            -0.2548   0.0646   -3.95         0.0

In [37]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

## Save and Report Model

If we are satisified with this model, or if we just want to record 
it as part of our workflow while exploring different model
structures, we can write the model out to a report.  To do so,
we can instantiatie a `larch.Reporter` object.

In [38]:
report = larch.Reporter(title=m.title)

Then, we can push section headings and report pieces into the
report using the "<<" operator.

In [39]:
report << '# Parameter Summary' << m.parameter_summary()

<xmle.Elem 'div' with 3 children>

In [40]:
report << "# Estimation Statistics" << m.estimation_statistics()

<xmle.Elem 'div' with 5 children>

In [41]:
report << "# Utility Functions" << m.utility_functions()

alt,formula
1,+ P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST
2,+ P.ASC_SR + P.InVehTime * X.AUTO_TIME + P.Cost * X('AUTO_COST*(0.5)') + P('LogIncome:SR') * X('log(INCOME)')
3,+ P.ASC_Walk + P.NonMotorTime * X.WALK_TIME + P('LogIncome:Walk') * X('log(INCOME)')
4,+ P.ASC_Bike + P.NonMotorTime * X.BIKE_TIME + P('LogIncome:Bike') * X('log(INCOME)')
5,+ P.ASC_Transit + P.InVehTime * X.TRANSIT_IVTT + P.OutVehTime * X.TRANSIT_OVTT + P.Cost * X.TRANSIT_FARE + P('LogIncome:Transit') * X('log(INCOME)')


Once we have assembled the report, we can save the file to 
disk as an HTML report containing the content previously 
assembled. Attaching the model itself to the report as
metadata can be done within the `save` method, which will
allow us to directly reload the same model again later.

In [42]:
report.save(
    '/tmp/exampville_mode_choice.html', 
    overwrite=True, 
    metadata=m,
)

'/tmp/exampville_mode_choice.html'

Note: if you get a `FileNotFound` error when saving, make sure that
you are saving the file into a directory that exists.  The example
here should work fine on macOS or Linux, but the `/tmp` directory
does not exist by default on Windows.

You can also save a model as an Excel file, which will automatically
include several worksheets summarizing the parameters, data, utility
functions, and other features of the model.

In [ ]:
import larch.util.excel
m.to_xlsx("/tmp/exampville_mode_choice.xlsx")